<a href="https://colab.research.google.com/github/gyuwonchoi/-CodingTest/blob/main/pytorch_with_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 파이토치 특징
* Tensor는 NumPy와 유사하지만 GPU 상에서 실행 가능
* AutoGrad를 활용한 자동 미분

이번 튜토리얼에서는 third order polynomial 3차 다항식을 활용하여 sin(x)에 근사하도록 함, 경사 하강법의 활용


### Tensor

우선 NumPy를 활용하여 신경망을 구성한 뒤 텐서에 대한 이해

* 넘파이는 computaion 그래프, gradient에 대하여 어려움

In [2]:
import numpy as np
import math 

# 랜덤 입출력
x = np.linspace(-math.pi, math.pi, 2000) # Return evenly spaced numbers over a specified interval
                                         # Stard, End, num of sample
y = np.sin(x) # G.T

# 가중치 초기화, 3차 다항식 이므로 계수 4개 필요 
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate= 1e-6

for t in range(2300):
    y_pred=a + b*x + c*x**2 + d*x**3

    loss = np.square(y_pred - y).sum() # 제곱으로 차
    if t%100==99:
      print(f"iter:{t} , loss: {loss}")

    # 그려보면 다 맞다 
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 가중치를 갱신합니다.
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

iter:99 , loss: 1534.637531517626
iter:199 , loss: 1068.696797078879
iter:299 , loss: 745.704968280226
iter:399 , loss: 521.5897919274852
iter:499 , loss: 365.93719621871134
iter:599 , loss: 257.73533681680493
iter:699 , loss: 182.45286095530923
iter:799 , loss: 130.0299441483298
iter:899 , loss: 93.4953982929824
iter:999 , loss: 68.01369206123735
iter:1099 , loss: 50.227518476460084
iter:1199 , loss: 37.80376669099
iter:1299 , loss: 29.11963736242585
iter:1399 , loss: 23.045420974630712
iter:1499 , loss: 18.794020188052915
iter:1599 , loss: 15.81660626828327
iter:1699 , loss: 13.730196691844386
iter:1799 , loss: 12.267341974275404
iter:1899 , loss: 11.241140826955444
iter:1999 , loss: 10.520892619197106
iter:2099 , loss: 10.015138610330823
iter:2199 , loss: 9.659840118499314
iter:2299 , loss: 9.410131189799584
Result: y = -0.02471361389809484 + 0.8634641835777135 x + 0.004263509633731696 x^2 + -0.09428672567991854 x^3


### Tensor


In [4]:
import torch
import math 

dtype = torch.float
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# AutoGrad를 위하여 requries_grad를 True로 : backward 하면 자동으로 a 객체 내 grad 로 미분값 계산
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # loss는 loss function이고 이에 대한 식을 기준으로 auto grad를 함
    # loss.backward로 표현 

    loss.backward()

    # weight 업데이트 시에는 gradient 계산 않으므로
    # 이걸 optimizer와 optim.step()으로 대체 
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 매 iter마다 gradient 새로 계산 
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
    
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 543.2364501953125
199 377.4659729003906
299 263.3881530761719
399 184.79641723632812
499 130.59274291992188
599 93.16914367675781
699 67.30377197265625
799 49.40827941894531
899 37.014617919921875
999 28.422710418701172
1099 22.46063995361328
1199 18.319625854492188
1299 15.440839767456055
1399 13.437801361083984
1499 12.042908668518066
1599 11.070743560791016
1699 10.392655372619629
1799 9.919336318969727
1899 9.58869457244873
1999 9.357573509216309
Result: y = 0.02272731438279152 + 0.8480797410011292 x + -0.0039208391681313515 x^2 + -0.0920984223484993 x^3


 optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수들에 대한 손실의 변화도를 계산합니다.
loss.backward()

    # optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
optimizer.step()
